In [2]:
import requests
from scripts.utils.helper import *
from scripts.utils.config import dbInfor
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy
import sys
from datetime import datetime, timedelta

con = get_mysql_connection()
query_ticker = "select distinct ticker from ticker_overview"
list_ticker = get_data_from_mysql(con, query_ticker)

In [3]:
## get date of x session before
def get_day_session_before(con, day, x):
    query = f"SELECT date FROM price_board WHERE ticker = 'FPT' AND date < '{day}' ORDER BY date DESC LIMIT {x}, 1"
    date = get_data_from_mysql(con, query)
    return str(date.iloc[0]['date'].date())

In [4]:
day = str((datetime.today() - timedelta(days=1)).date())

In [5]:
day_65 = get_day_session_before(con, day, 65)
day_130 = get_day_session_before(con, day, 130)
day_260 = get_day_session_before(con, day, 260)

In [6]:
query = f"""
    SELECT ticker,
        max(CASE WHEN date = '{day}' THEN close END) as c,
        max(CASE WHEN date = '{day_65}' THEN close END) AS c_65,
        max(CASE WHEN date = '{day_130}' THEN close END) AS c_130,
        max(CASE WHEN date = '{day_260}' THEN close END) AS c_260
    FROM price_board
    where date = '{day}' or date = '{day_65}' or date = '{day_130}' or date = '{day_260}'
    group by ticker
    """
rsidb = get_data_from_mysql(con, query)

In [7]:
list_ticker = get_data_from_mysql(con,"""select distinct t1.ticker from (price_board as t1 join (select ticker from ticker_overview where comGroupCode = "VNINDEX") as t2 on t1.ticker = t2.ticker)""")

In [8]:
rsidb = rsidb.dropna(subset=['c', 'c_65', 'c_130', 'c_260'])

In [9]:
rsidb['roc_65'] = (rsidb['c'] - rsidb['c_65']) / rsidb['c_65']
rsidb['roc_130'] = (rsidb['c'] - rsidb['c_130']) / rsidb['c_130']
rsidb['roc_260'] = (rsidb['c'] - rsidb['c_260']) / rsidb['c_260']
rsidb['score'] = 0.4 * rsidb['roc_65'] + 0.3 * rsidb['roc_130'] + 0.3 * rsidb['roc_260']

In [10]:
df_all = rsidb.merge(list_ticker, on='ticker')

In [11]:
df_all['rank'] = df_all['score'].rank()

In [12]:
n = len(df_all)

In [13]:
df_all['rank'] = df_all['rank'].apply(lambda x: int((x-1)/n * 100))

In [14]:
eps_growth_query = """
    select t1.ticker, (eps - pre_eps) / pre_eps * 100 as growth_eps from
(select ticker, max(year) as year, max(quarter) as quarter, earningPerShare as eps
from financial_ratio
where quarter < 5
group by ticker) as t1
join 
(select ticker,year, quarter, earningPerShare as pre_eps
from financial_ratio
) as t2
on t2.year = t1.year -1 and t1.quarter = t2.quarter and t1.ticker = t2.ticker
"""

In [15]:
eps_growth = get_data_from_mysql(con, eps_growth_query)

In [18]:
df_all = df_all.merge(eps_growth[eps_growth['growth_eps'] >= 20], on='ticker')

In [19]:
profit_growth_query = """
select t1.ticker, (profit - pre_profit) / pre_profit * 100 as growth_profit from
(select ticker, max(year) as year, postTaxProfit as profit
from income_statement
where quarter = 5
group by ticker) as t1
join
(select ticker, year, postTaxProfit as pre_profit
from income_statement
where quarter = 5) as t2
on t2.year = t1.year - 3 and t1.ticker = t2.ticker
"""

In [20]:
profit_growth = get_data_from_mysql(con, profit_growth_query)

In [23]:
df_all = df_all.merge(profit_growth[profit_growth['growth_profit'] > 25], on='ticker')

In [25]:
roe_query = """
select ticker, max(year) as year, max(quarter) as quarter, roe
from financial_ratio
group by ticker
"""
roe = get_data_from_mysql(con, roe_query)

In [27]:
df_all = df_all.merge(roe[roe['roe'] >= 0.17], on='ticker')

In [28]:
df_all

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank,growth_eps,growth_profit,year,quarter,roe
0,CMV,9000.0,7800.0,13396.0,13166.0,0.153846,-0.328158,-0.316421,-0.131835,36,29.947230,27.2727,2022,5,0.331
1,HAG,7920.0,5930.0,11950.0,12200.0,0.335582,-0.337238,-0.350820,-0.072185,51,389.898990,1266.6667,2022,5,0.196
2,HRC,53300.0,57000.0,57000.0,54000.0,-0.064912,-0.064912,-0.012963,-0.049327,57,426.438356,900.0000,2022,5,0.425
3,ITA,4130.0,2590.0,8150.0,15500.0,0.594595,-0.493252,-0.733548,-0.130202,36,29.677419,32.5301,2022,5,0.196
4,ITD,12050.0,10500.0,12250.0,14800.0,0.147619,-0.016327,-0.185811,-0.001594,68,78.382148,30.2326,2022,5,0.284
5,QCG,4000.0,3320.0,8950.0,12600.0,0.204819,-0.553073,-0.682540,-0.288756,12,31.914894,27.7228,2022,5,0.180
6,SJS,45100.0,47200.0,69800.0,82000.0,-0.044492,-0.353868,-0.450000,-0.258957,15,300.936768,52.6316,2022,5,0.360
7,SPM,56325.0,13041.0,16808.0,18794.0,3.319071,2.351083,1.996967,2.632043,99,483.754266,122.2222,2022,5,0.519
8,VAF,8582.0,8700.0,13200.0,12210.0,-0.013563,-0.349848,-0.297133,-0.199520,23,199.056604,341.6667,2022,5,0.187
